In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [3]:
import pickle

with open('URM_train_validation.pkl', 'rb') as f:
    URM_train_validation = pickle.load(f)

with open('URM_test.pkl', 'rb') as f:
    URM_test = pickle.load(f)

In [6]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = SLIMElasticNetRecommender(URM_train_validation)
slim.load_model("slim_models/", "slim_t_val")
## P3ALPHA
p3alpha = P3alphaRecommender(URM_train_validation)
p3alpha.fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
## RP3BETA
rp3beta = RP3betaRecommender(URM_train_validation)
rp3beta.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
## USER KNN
userknn = UserKNNCFRecommender(URM_train_validation)
userknn.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
## ITEM KNN
itemknn = ItemKNNCFRecommender(URM_train_validation)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
## EASE_R
ease_r = EASE_R_Recommender(URM_train_validation)
ease_r.load_model("slim_models/", "ease_t_val")

model = LinearHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'slim_models/slim_t_val'
SLIMElasticNetRecommender: Loading complete
P3alphaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 7190.98 column/sec. Elapsed time 3.09 sec
RP3betaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 6756.19 column/sec. Elapsed time 3.29 sec
UserKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
Similarity column 12638 (100.0%), 9785.54 column/sec. Elapsed time 1.29 sec
ItemKNNCFR

In [8]:
##HYBRIDS
knn = TwoScoresHybridRecommender(URM_train_validation, userknn, itemknn)
knn.fit(alpha=0.022195783788315104)
superH = LinearHybridRecommender(URM_train_validation, [slim, rp3beta, knn, ease_r])
superH.fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])

hybrid_new = LinearHybridRecommender(URM_train_validation, [slim, rp3beta, itemknn, ease_r])
hybrid_new.fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])

hybrid_old = ScoresHybridRecommender(URM_train_validation, slim, rp3beta, itemknn)
hybrid_old.fit(alpha=0.5072513954300486, beta=0.30902582824263064)

hybrid_hi = TwoScoresHybridRecommender(URM_train_validation, slim, rp3beta)
hybrid_hi.fit(alpha=0.6201320790279279)

graph_h = TwoScoresHybridRecommender(URM_train_validation, p3alpha, rp3beta)
graph_h.fit(alpha=0.14935090288050779)

UserKNNCFItemKNNCFTwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFItemKNNCFTwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaItemKNNCFEASE_R_LinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFEASE_R_LinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaItemKNNCFScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaTwoScoresHybridRecommender: URM Detected 205 ( 1.6%) user

In [7]:
ease = ease_r

In [9]:
import itertools
import optuna as op

# Map the names to the recommender objects
recommender_dict = {
    'slim': slim,
    'p3alpha': p3alpha,
    'rp3beta': rp3beta,
    'userknn': userknn,
    'itemknn': itemknn,
    'easer': ease,
    'knn': knn,
    'super': superH,
    'hybridnew': hybrid_new,
    'hybridold': hybrid_old,
    'hybridhi': hybrid_hi,
    'graph': graph_h
}

# Generate all possible combinations of recommender names
all_combinations = []
for r in range(1, len(recommender_dict) + 1):
    all_combinations.extend(itertools.combinations(recommender_dict.keys(), r))

# Convert combinations to string format
all_combinations_str = ['_'.join(comb) for comb in all_combinations]

def objective(trial):
    # Select a combination from the pre-defined list
    selected_combination_str = trial.suggest_categorical('selected_combination', all_combinations_str)

    # Convert the string back to the original recommender objects
    selected_combination = [recommender_dict[name] for name in selected_combination_str.split('_')]

    # Dynamically generate weights for the selected recommenders
    weights = [trial.suggest_float(f'weight_{i}', 0, 1) for i in range(len(selected_combination))]
    
    # Create hybrid recommender with the suggested weights
    hybrid_recommender = model(URM_train_validation, selected_combination)
    hybrid_recommender.fit(weights)
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "alphas": 1,
}

study_name = "hybrid-allmodel2s-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db".format(study_name)
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=150)

In [12]:
study.best_params

{'selected_combination': 'slim_easer_super_hybridold_graph',
 'weight_0': 0.6748705047734029,
 'weight_1': 0.12944639345936737,
 'weight_2': 0.6039430610485853,
 'weight_3': 0.8168483325375601,
 'weight_4': 0.2978584217886625}

In [13]:
hybrid_recommender = LinearHybridRecommender(URM_train_validation, [slim, ease, superH, hybrid_old, graph_h])

SLIMElasticNetEASE_R_SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridSLIMElasticNetRP3betaItemKNNCFScoresHybridP3alphaRP3betaTwoScoresHybridLinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetEASE_R_SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridSLIMElasticNetRP3betaItemKNNCFScoresHybridP3alphaRP3betaTwoScoresHybridLinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [14]:
def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 1)
    beta = trial.suggest_float('beta', 0, 1)
    gamma = trial.suggest_float('gamma', 0, 1)
    delta = trial.suggest_float('delta', 0, 1)
    epsilon = trial.suggest_float('epsilon', 0, 1)
    
    # Create hybrid recommender with the suggested weights
    hybrid_recommender.fit(alphas=[alpha, beta, gamma, delta, epsilon])
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid_recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
study_name = "hybrid-mixed-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=150)

[I 2024-01-10 23:04:13,746] A new study created in RDB with name: hybrid-mixed-study


EvaluatorHoldout: Processed 10435 (100.0%) in 16.39 sec. Users per second: 637


[I 2024-01-10 23:04:30,208] Trial 0 finished with value: 0.05168819449198045 and parameters: {'alpha': 0.533294175974392, 'beta': 0.814664103181865, 'gamma': 0.8052948702291137, 'delta': 0.35083807880822004, 'epsilon': 0.7791298625987269}. Best is trial 0 with value: 0.05168819449198045.


EvaluatorHoldout: Processed 10435 (100.0%) in 17.43 sec. Users per second: 599


[I 2024-01-10 23:04:47,675] Trial 1 finished with value: 0.051602961644649045 and parameters: {'alpha': 0.28585587875451024, 'beta': 0.5097292753768967, 'gamma': 0.35077193972702947, 'delta': 0.3059751036939399, 'epsilon': 0.09450148056163687}. Best is trial 0 with value: 0.05168819449198045.


...

[I 2024-01-10 23:37:03,776] Trial 121 finished with value: 0.05205259695317247 and parameters: {'alpha': 0.5886035289467368, 'beta': 0.12022440503091614, 'gamma': 0.8997057033749583, 'delta': 0.1057773039962644, 'epsilon': 0.36757545283081267}. Best is trial 3 with value: 0.05209182695598662.


EvaluatorHoldout: Processed 10435 (100.0%) in 14.41 sec. Users per second: 724


[I 2024-01-10 23:37:18,240] Trial 122 finished with value: 0.05200127775115856 and parameters: {'alpha': 0.544412553546788, 'beta': 0.13277927477756163, 'gamma': 0.8724939422671527, 'delta': 0.11180866159532277, 'epsilon': 0.41767712472562374}. Best is trial 3 with value: 0.05209182695598662.


EvaluatorHoldout: Processed 10435 (100.0%) in 14.01 sec. Users per second: 745


[I 2024-01-10 23:37:32,307] Trial 123 finished with value: 0.05199764604771844 and parameters: {'alpha': 0.5811630172587513, 'beta': 0.11777057799981264, 'gamma': 0.8076512102763975, 'delta': 0.04861366209419293, 'epsilon': 0.35472130986450245}. Best is trial 3 with value: 0.05209182695598662.


In [16]:
study.best_params

{'alpha': 0.8857667100747117,
 'beta': 0.6379807942443021,
 'gamma': 0.0690929184825888,
 'delta': 0.11953478623354052}